# 📌 Introduction

In this notebook, I will be working with the Online Retail Data Set from Kaggle (uploaded by vijayuv) https://www.kaggle.com/datasets/vijayuv/onlineretail. 🛒

This dataset contains transactional records from a UK-based online retail store, including:

- 🧾 InvoiceNo – transaction ID

- 📦 StockCode – product code

- 📝 Description – product description

- 🔢 Quantity – number of items

- 💲 UnitPrice – price per item

- 🧑‍🤝‍🧑 CustomerID – unique customer identifier

- 🌍 Country – customer location

👉 The main goal here is not deep analysis, but rather data wrangling 🔧:

- 🧐 Inspect the raw data structure

- 🧹 Handle missing values & duplicates

- ⏳ Correct data types

- ✍️ Clean inconsistencies in product descriptions and customer IDs

- 📂 Prepare a clean dataset ready for analysis

✨ By the end of this process, we’ll have a consistent, reliable, and tidy dataset that can be used for further exploration and analytics.

## 📦 0. Install Library

In [1]:
!pip install pandas numpy

## 📦 1. Import Library

In [2]:
import pandas as pd
import numpy as np
from google.colab import files
import io
from datetime import datetime

## 📥 2. Upload File from Kaggle

In [3]:
uploaded = files.upload()

Saving OnlineRetail.csv to OnlineRetail.csv


## 📊 3. Extract file name

In [4]:
# Ambil nama file dari dict
filename = next(iter(uploaded))

print(filename)

OnlineRetail.csv


## 📊 4. Load Dataset

In [5]:
df = pd.read_csv(filename, encoding='latin1')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


From the first 5 rows of the dataset, we can understand its overall structure and the types of data it contains. The dataset includes several key columns such as InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, and Country.

* InvoiceNo is a transaction identifier, categorical in nature, used to distinguish each invoice or purchase record.

* StockCode is a unique product code, also categorical.

* Description contains free-text product descriptions, which are textual data.

* Quantity and UnitPrice are numerical fields (integer and float) that can be used to calculate transaction amounts and analyze purchase volumes.

* InvoiceDate is a datetime field (time series) that enables analysis of sales trends across different time periods, seasons, or specific dates.

* CustomerID is a unique customer identifier, crucial for understanding customer behavior, retention, and segmentation.

* Country is a categorical field showing the customer’s country, useful for geographic sales distribution analysis.

Overall, this dataset contains time series data, numerical data (integers and floats), categorical data, and textual data. Recognizing these types of data is essential for planning data cleaning, handling missing values, and choosing the right analytical or modeling approaches.

## 🔍 5. Initial Exploration

In [6]:
df.info()
print(df.nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB
InvoiceNo      25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
UnitPrice       1630
CustomerID      4372
Country           38
dtype: int64


Key observations:
- 'InvoiceNo', 'StockCode', 'Description', 'InvoiceDate', and 'Country' are currently stored as object types.
- 'InvoiceDate' has 23,260 unique values, but is in string format and should be converted to datetime for time-based analysis.
- 'CustomerID' is numeric (float64) but contains missing values, as seen in earlier checks.
- Numerical columns include 'Quantity' and 'UnitPrice'.
- 'Country' has 38 unique values, indicating international transactions.

Next step:
- Convert 'InvoiceDate' from object to datetime for accurate date filtering, grouping, and trend analysis.

In [7]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df = df.set_index('InvoiceDate')

In [8]:
df.describe(include='all')

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country
count,541909,541909,540455,541909.000000,541909.000000,406829.000000,541909
unique,25900,4070,4223,NaN,NaN,NaN,38
top,573585,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,NaN,NaN,United Kingdom
freq,1114,2313,2369,NaN,NaN,NaN,495478
mean,NaN,NaN,NaN,9.552250,4.611114,15287.690570,NaN
std,NaN,NaN,NaN,218.081158,96.759853,1713.600303,NaN
min,NaN,NaN,NaN,-80995.000000,-11062.060000,12346.000000,NaN
25%,NaN,NaN,NaN,1.000000,1.250000,13953.000000,NaN
50%,NaN,NaN,NaN,3.000000,2.080000,15152.000000,NaN
75%,NaN,NaN,NaN,10.000000,4.130000,16791.000000,NaN


* Most frequent product is WHITE HANGING HEART T-LIGHT HOLDER (2,369 times), and the most frequent country is United Kingdom (495,478 transactions).

* Quantity ranges from -80,995 to 80,995, with negative values indicating returns or cancellations. The median quantity is 3, meaning most transactions are small.

* UnitPrice ranges from -11,062.06 to 38,970, with negative values likely due to cancellations or adjustments. The median unit price is 2.08.

* CustomerID has 4,372 unique values.

## 🧽 6. Check Missing & Duplicate Values

In [9]:
print("Missing Values per Column:\n", df.isnull().sum())
print("Number of duplicates:", df.duplicated().sum())

Missing Values per Column:
 InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64
Number of duplicates: 5269


From the missing values and duplicate check:

* Description column has 1,454 missing values, which may indicate incomplete product information.

* CustomerID column has a significant 135,080 missing values.

* 5,268 duplicate rows were found, which could affect analysis accuracy if not handled properly.

Recommendations:
- For 'Description':
  If product details are critical to the analysis, remove rows where 'Description' is missing
  using df.dropna(subset=['Description']), or fill with a placeholder such as 'Unknown'.

- For 'CustomerID':
  Even though this is still the data wrangling stage, drop rows with missing CustomerID
  using df.dropna(subset=['CustomerID']) to ensure that all transactions are tied to a customer.

- For duplicates:
  Remove duplicate rows using df.drop_duplicates() to ensure accurate analysis.

## 7. Treat Missing Values

In [10]:
# Fill missing values in 'Description'
df['Description'] = df['Description'].fillna('Unknown')

# Drop rows with missing values in 'CustomerID'
df.dropna(subset=['CustomerID'], inplace=True)

print("\nNumber of Missing Values after handling:")
print(df.isnull().sum())


Number of Missing Values after handling:
InvoiceNo      0
StockCode      0
Description    0
Quantity       0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64



Actions Taken:
- For 'Description':
  Missing values were filled with the placeholder 'Unknown' to preserve transaction data
  while maintaining completeness in the column.

- For 'CustomerID':
  Rows with missing values were dropped because:
    1. These transactions cannot be tied to a specific customer, making them unusable for customer-level analysis.
    2. Keeping them could bias customer metrics (e.g., average spend, frequency, segmentation results).
    3. Filling them with placeholders like 'Guest' would not add meaningful information,
       since the true identity is unknown.
    4. Dropping ensures all remaining transactions are linked to valid customer IDs,
       resulting in cleaner and more reliable analysis.

Outcome:
After handling missing values, the dataset now contains 0 missing values across all columns,
ensuring readiness for the next steps in data wrangling and analysis.

## 8. Handle Duplicates

In [11]:
df.drop_duplicates(inplace=True)

In [12]:
duplicate_rows_count = df.duplicated().sum()
print(f"Number of duplicate rows in the dataset: {duplicate_rows_count}")

Number of duplicate rows in the dataset: 0


For duplicates:
  Rows will be removed to ensure accurate metrics and prevent inflated counts.

## 9. Feature Engineering

In [13]:
# Total transaction per row
df['TotalAmount'] = df['Quantity'] * df['UnitPrice']
df.head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,TotalAmount
InvoiceDate,,,,,,,,
2010-12-01 08:26:00,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,15.30
2010-12-01 08:26:00,536365,71053,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,20.34
2010-12-01 08:26:00,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom,22.00
2010-12-01 08:26:00,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom,20.34
2010-12-01 08:26:00,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom,20.34


'TotalAmount' was created by multiplying 'Quantity' and 'UnitPrice',
representing the value of each transaction line.

This feature is essential for:
- Calculating revenue at various levels (transaction, customer, time period).
- Analyzing sales trends, customer spending, and business performance.

## 10. Save New Data to Excel

In [14]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Save DataFrame to Excel
df.to_excel('Gabriella_OnlineRetail.xlsx', index=False)

# 3. Copy to Google Drive folder (for example, to "My Drive/Dataset")
!cp Gabriella_OnlineRetail.xlsx "/content/drive/My Drive/Dataset/"

Mounted at /content/drive
